# report

In [1]:
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from contract_select import contract_select

In [ ]:
data= 'spy'
date= '20170912'
maxDTE: int = 30,
minDTE: int = 10,
maxDistance: float = 0.2,
minDistance: float = 0.08,
maxTheta: float = -0.0,
minTheta: float = -1.2,
contract_type: str = 'C'
moneyness = 'out'

contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                   index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
contract_pd = contract_pd.drop(columns='Unnamed: 0')
contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]

In [ ]:
contract_select(selct_type='main', date= '20190913', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)[1]

In [ ]:
contract_select(selct_type='hedge', date= '20190913', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)[1]

In [ ]:
import pandas as pd
import os

def greek_calc(
    position: pd.DataFrame = None,
    ) ->pd.DataFrame:
    
"""
return a dataframe containing the current greeks exposure

input:
position: current position
"""


def pnl_calc(
    position: pd.DataFrame = None,
    ) ->pd.DataFrame:
    
"""
return a dataframe containing the pnl comparing to last trading day

input:
position: current position
"""


In [2]:
df1=contract_select(selct_type='main', date= '20190916', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)
df2=contract_select(selct_type='hedge', date= '20190916', spy_spx='spx', contract_type='C', minAsk=0.2, minDistance = 0.03, num=3)

In [5]:
import pandas as pd
import os

def hedge_calc(
    main: pd.DataFrame = None,
    hedge: pd.DataFrame = None,
    main_data: str = None,
    hedge_data: str = None
    ) ->pd.DataFrame:
    
    """
    return a dataframe containing the information of selected contrats
    
    input:
    main: a dataframe containing the main contract to hedge
    hedge: a dataframe containing the hedge contract to hedge
    main_data: type of contract, spx or spy
    hedge_data: type of contract, spx or spy
    """

    if main_data == 'spy':
        columns = main.filter(regex='DELTA').columns
        main[columns] = main[columns]/10
        # columns = main.filter(regex='GAMMA').columns
        # main[columns] = main[columns]/10
        # columns = main.filter(regex='THETA').columns
        # main[columns] = main[columns]*10
    if hedge_data == 'spy':
        columns = hedge.filter(regex='DELTA').columns
        hedge[columns] = hedge[columns]/10
        # columns = main.filter(regex='GAMMA').columns
        # hedge[columns] = hedge[columns]/10
        # columns = hedge.filter(regex='THETA').columns
        # hedge[columns] = hedge[columns]*10
    
    # Extract columns whose names contain the desired string
    desired_string = ['GAMMA', 'VEGA', 'THETA']
    main_filtered_columns = []
    hedge_filtered_columns = []
    for i in desired_string:
        main_filtered_columns.append([col for col in main.columns if i in col])
        hedge_filtered_columns.append([col for col in hedge.columns if i in col])
        
    #flatten the column list
    main_filtered_columns=sum(main_filtered_columns,[])
    hedge_filtered_columns= sum(hedge_filtered_columns,[])
    main_mean = main[main_filtered_columns].values.mean(axis=0)
    main_mean_target = main_mean.copy()
    main_mean_target[-1] = 0
    
    def fun(x):
        # x = np.array(x)
        utility = np.absolute(x.dot(hedge[hedge_filtered_columns].values) - main_mean_target)
        #add weights to the residual greeks after hedging 
        utility = utility * np.array([[50],[1],[10]])
        r=np.sum(utility)
        return r
    
    def fun1(x):
        greeks = x.dot(hedge[hedge_filtered_columns].values) - main_mean
        return greeks

    constraint1 = LinearConstraint(np.array(hedge[hedge_filtered_columns[-1]].values), lb= main_mean[-1]*0.2, ub=[np.inf])
    constraint2 = NonlinearConstraint(lambda x: np.sum(np.absolute(x)), lb=0, ub=1, keep_feasible=True)
    constraint3 = LinearConstraint(np.array(hedge[hedge_filtered_columns[0]].values), lb = main_mean_target[0] - abs(main_mean_target[0])*0.2, ub = main_mean_target[0] + abs(main_mean_target[0])*0.2)
    constraint4 = LinearConstraint(np.array(hedge[hedge_filtered_columns[1]].values), keep_feasible=True, lb= main_mean_target[1] - abs(main_mean_target[1])*0.2, ub = main_mean_target[1] + abs(main_mean_target[1])*0.2)
    
    x0 = [min(0.1/len(hedge),main_mean[1]/(hedge[hedge_filtered_columns[1]].mean()*len(hedge)))]*len(hedge)
    result = minimize(fun, x0, method='trust-constr', options={'maxiter':1000}, tol=1e-6)
    hedge['WEIGHT'] = result.x
    main['WEIGHT'] = -1/len(main)
    
    x=result.x
    
    #calculate the emini weight
    quote_date = main.index[0][0]
    main_delta = sum(main.filter(like='DELTA').values * main.filter(like='WEIGHT').values)
    hedge_delta = sum(hedge.filter(like='DELTA').values * hedge.filter(like='WEIGHT').values)
    emini =  pd.read_csv('emini/emini_eod_'+ quote_date[:4]+ quote_date[5:7] +'.csv', skipinitialspace=True)
    emini = emini[emini['[QUOTE_DATE]'] == quote_date].set_index(['[QUOTE_DATE]'])
    emini['WEIGHT'] = -main_delta-hedge_delta
    
    result = {'main':main, 'main_type':main_data, 'hedge': hedge, 'hedge_type':hedge_data, 'emini':emini}
    return result

In [106]:
emini =  pd.read_csv('emini/emini_eod_201909.csv', skipinitialspace=True)
emini[emini['[QUOTE_DATE]'] ==quote_date].set_index(['[QUOTE_DATE]'])

,[UNDERLYING_LAST]
[QUOTE_DATE],
2019-09-16,2997.8


In [ ]:
x=np.array([ 0.00762753, -0.42346308,  0.04095649])

In [ ]:
np.dot(np.array([[0.1],[1],[0]]),np.array([[50,1,10]]))

In [31]:
quote_date=df2[1].index[0][0]

In [37]:
quote_date[5:7]

'09'

In [6]:
hedge_calc(df1[1],df2[1],df1[0][:3],df2[0][:3])

{'main':                             [UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                                
 2019-09-16   2019-10-02                2997.8   16.0    2705.0     1.46   
              2019-10-02                2997.8   16.0    2725.0     1.71   
              2019-10-04                2997.8   18.0    2710.0     1.91   
 
                             [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                            
 2019-09-16   2019-10-02        1.54   -0.02492    0.00034   0.37644   
              2019-10-02        1.80   -0.02941    0.00044   0.43293   
              2019-10-04        2.00   -0.03073    0.00043   0.47192   
 
                             [P_THETA]  [STRIKE_DISTANCE]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                 
 2019-09-16   2019-10-02      -0.25808              292.8   
              2019-10-02      -0.28790              272.8   
    

In [53]:
d[4]

[UNDERLYING_LAST]  [DTE]  [C_DELTA]  [C_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                   
2019-09-16   2019-10-16                2997.8   30.0    0.14180    0.00260   
             2019-10-16                2997.8   30.0    0.12793    0.00241   
             2019-10-16                2997.8   30.0    0.11638    0.00230   

                            [C_VEGA]  [C_THETA]  [C_BID]  [C_ASK]  [STRIKE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                    
2019-09-16   2019-10-16      1.97473   -0.30050     5.99      6.3    3090.0   
             2019-10-16      1.83691   -0.27663     5.20      5.5    3095.0   
             2019-10-16      1.71716   -0.25842     4.61      4.9    3100.0   

                            [STRIKE_DISTANCE]  [STRIKE_DISTANCE_PCT]  \
[QUOTE_DATE] [EXPIRE_DATE]                                             
2019-09-16   2019-10-16                  92.2                  0.031   
             2019-10-16                  97.2                  0.032   
             2019-10-16                 102.2                  0.034   

                                utility    weight  
[QUOTE_DATE] [EXPIRE_DATE]                         
2019-09-16   2019-10-16    -1627.837189  0.080180  
             2019-10-16    -1498.934073  0.076994  
             2019-10-16    -1390.179523  0.074154

In [58]:
d[4].loc['2019-09-16']

,[UNDERLYING_LAST],[DTE],[C_DELTA],[C_GAMMA],[C_VEGA],[C_THETA],[C_BID],[C_ASK],[STRIKE],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT],utility,weight
[EXPIRE_DATE],,,,,,,,,,,,,
2019-10-16,2997.8,30.0,0.14180,0.00260,1.97473,-0.30050,5.99,6.3,3090.0,92.2,0.031,-1627.837189,0.080180
2019-10-16,2997.8,30.0,0.12793,0.00241,1.83691,-0.27663,5.20,5.5,3095.0,97.2,0.032,-1498.934073,0.076994
2019-10-16,2997.8,30.0,0.11638,0.00230,1.71716,-0.25842,4.61,4.9,3100.0,102.2,0.034,-1390.179523,0.074154


In [ ]:
np.array([0.00789847, -0.41962886, -0.04234271]).dot(np.array([0.00838, 0.00769, 0.00590]))

In [ ]:
0.00789847*0.00838+-0.41962886*0.00769-0.04234271*0.00590

In [ ]:
def fun(x):
    x = np.array(x)
    utility = np.absolute(x.dot(np.array([[0.008, 1.07542,-0.37072],[0.00071,0.98295,-0.34664],[0.0007,1.04561,-0.36438]]))-np.array([.0004,0.51869,0]))
    utility1 = utility \
        * np.array([[50],[1],[10]])
    r=np.sum(utility1)*10
    return r

def fun1(x):
    x = np.array(x)
    utility = np.absolute(x.dot(np.array([[0.008, 1.07542,-0.37072],[0.00071,0.98295,-0.34664],[0.0007,1.04561,-0.36438]]))-np.array([.0004,0.51869,0]))
    utility1 = utility * np.array([[1e2],[1],[10000]])
    return utility

constraint1 = LinearConstraint(np.array([-0.37072, -0.34664, -0.36438]), lb=-0.05, ub=0.05)
constraint2 = NonlinearConstraint(lambda x: np.sum(np.absolute(x)), lb=0, ub=1)
constraint3 = LinearConstraint(np.array([0.008, 0.00071, 0.0007]), lb=0.0004-0.0004*0.5, ub=0.0004+0.004*0.5)
constraint4 = LinearConstraint(np.array([1.07542, 0.98295, 1.04561]), lb=0.57869-0.57869*0.5, ub=0.57869+0.57869*0.5)

x0 = [-0.1,0.1,0.1]
result = minimize(fun, x0, method='trust-constr', options={'maxiter':1000}, constraints={constraint1,constraint2,constraint3,constraint4}, tol=1e-4)

In [ ]:
x=result.x
fun1(x)